In [3]:
import pickle
import pandas as pd

with open("../models/als_model.pkl", "rb") as f:
    als_model = pickle.load(f)
with open("../models/user_mapping.pkl", "rb") as f:
    user_encoder = pickle.load(f)
with open("../models/items_mapping.pkl", "rb") as f:
    item_encoder = pickle.load(f)
with open("../models/tfidf_vectorizer.pkl", "rb") as f:
    tfidf = pickle.load(f)
with open("../models/item_index.pkl", "rb") as f:
    item_index = pickle.load(f)
with open("../models/product_text.pkl", "rb") as f:
    product_text = pickle.load(f)

In [4]:
from sklearn.metrics.pairwise import cosine_similarity
tfidf_matrix = tfidf.transform(product_text["clean_text"])
def content_scores_for_items(base_asin, candidate_asin):
    if base_asin not in item_index:
        return{}

    base_idx = item_index[base_asin]
    base_vec = tfidf_matrix[base_idx]
    scores = {}
    for asin in candidate_asins:
        if asin in item_index:
            idx = item_index[asin]
            sim = cosine_similarity(base_vec, tfidf_matrix[idx])[0][0]
            scores[asin] = sim
    return scores
    

In [5]:
def hybrid_recommend(user_id_raw, N=10):
    recommendations = {}
    if user_id_raw in user_encoder.classes_:
        user_id = user_encoder.transform([user_id_raw])[0]
        item_ids, cf_scores = als_model.recommend(
            user_id,
            interactions[user_id],
            N=50
        )
        asins = item_encoder.inverse_transform(item_ids)
        cf_scores = (cf_scores - cf_scores.min()) / (cf_scores.max() - cf_scores.min() + 1e-8)
        cf_dict = dict(zip(asins, cf_scores))
        seed_asin = asins[0]
        content_dict = content_scores_for_items(seed_asin, asins)
        for asin in asins:
            recommendations[asin] = (
                0.7 * cf_dict.get(asin, 0) +
                0.3 * content_dict.get(asin, 0)
            )
    else:
        popular_items = (
            df.groupby("asin")["overall"]
            .mean()
            .sort_values(ascending=False)
            .head(N)
            .index
            .tolist()
        )
        return popular_items
    ranked = sorted(
        recommendations.items(),
        key=lambda x: x[1],
        reverse=True
    )
    return [asin for asin, _ in ranked[:N]]